<a href="https://colab.research.google.com/github/giuliacern/HyperfMNIST/blob/master/Iris_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install GPyOpt
!pip uninstall tensorflow
!pip install tensorflow==2.0.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 61kB 1.0MB/s 
     |████████████████████████████████| 1.0MB 3.1MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-cp36-none-any.whl size=83623 sha256=7d7b746cbed9b6b0e22d732de77f8402d5153f14ba70994e338afc5a83a8c4da
  Stored in directory: /root/.cache/pip/wheels/b2/00/69/cfa967a125cf25e66f644be6193ad6f0edf231147879ad714f
  Created wheel for GPy: filename=GPy-1.9.9-cp36-cp36m-linux_x86_64.whl size=2633939 sha256=ec542c752847e38b85f48dda252568100323bd29da8d39192c6ec2bceeee0402
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp36-none-any.whl size=102552 sha256=5d5bbe391b2a67a648d18e772a1c38ab2314816981e92263adc7d4f38e942846
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPyOpt GPy paramz
Uninstalling tensor

In [0]:
import GPy, GPyOpt
import numpy as np
import pandas as pd
import random

from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [0]:
# IRIS class
class IRIS():
    def __init__(self, l1, first_input=4, last_output=3,
                 batch_size=10, 
                 epochs=10, 
                 validation_split=0.1):
        self.__l1 = l1
        self.__first_input = first_input
        self.__last_output = last_output
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.iris_data()
        self.__model = self.iris_model()
        
    def iris_data(self):
        iris_data = load_iris() # load the iris dataset
        x = iris_data.data
        y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column
        # One Hot encode the class labels
        encoder = OneHotEncoder(sparse=False)
        y = encoder.fit_transform(y_)
        # Split the data for training and testing
        X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)
        return X_train, X_test, Y_train, Y_test
    
    
    
    def iris_model(self):
        model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(self.__l1, input_shape=(self.__first_input,), activation=tf.nn.relu),
        tf.keras.layers.Dense(self.__last_output, activation=tf.nn.softmax),
        ])
        
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model
    
    def iris_fit(self):        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split)
    
    def iris_evaluate(self):
        self.iris_fit()   
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [0]:
def run_iris(l1=512, first_input=4, last_output=3,
              batch_size=10, epochs=1, validation_split=0.1):
    
    _iris = IRIS(l1 = l1, first_input=first_input, last_output=last_output,
                   batch_size=batch_size, epochs=epochs, 
                   validation_split=validation_split)
    iris_evaluation = _iris.iris_evaluate()
    return iris_evaluation

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# the bounds dict should be in order of continuous type and then discrete type
domain = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.1, 0.3)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (10, 50)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (5, 10, 20)},
          {'name': 'l1',               'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)}
]

def f(x):
    print("values on domain: ", x)
    evaluation = run_iris(
        batch_size = int(x[:,1]), 
        epochs = int(x[:,2]), 
        l1 = int(x[:,3]),
        validation_split = float(x[:,0]))
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    print("\n")
    return evaluation[0]

# optimizer
opt_iris = GPyOpt.methods.BayesianOptimization(f=f, domain=domain)
opt_iris.run_optimization(max_iter=10)
        
opt_iris.plot_acquisition()

values on domain:  [[ 0.10199806 50.         10.         64.        ]]
LOSS:	1.1248494386672974 	 ACCURACY:	0.30000001192092896
[1.1248494386672974, 0.3]


values on domain:  [[  0.1630928  10.          5.        128.       ]]
LOSS:	0.7495896021525065 	 ACCURACY:	0.6000000238418579
[0.7495896021525065, 0.6]


values on domain:  [[  0.19555108  10.          10.         128.        ]]
LOSS:	0.5148189067840576 	 ACCURACY:	1.0
[0.5148189067840576, 1.0]


values on domain:  [[2.47300516e-01 5.00000000e+01 2.00000000e+01 1.02400000e+03]]
LOSS:	0.39376893639564514 	 ACCURACY:	0.8999999761581421
[0.39376893639564514, 0.9]


values on domain:  [[  0.2183978  10.         10.        128.       ]]
LOSS:	0.504764715830485 	 ACCURACY:	0.699999988079071
[0.504764715830485, 0.7]


values on domain:  [[2.46504962e-01 5.00000000e+01 1.00000000e+01 1.02400000e+03]]
LOSS:	0.5537739992141724 	 ACCURACY:	0.7333333492279053
[0.5537739992141724, 0.73333335]


values on domain:  [[2.40826566e-01 5.00000000e+01

In [0]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(domain[0]["name"], opt_iris.x_opt[0],
           domain[1]["name"], opt_iris.x_opt[1],
           domain[2]["name"], opt_iris.x_opt[2],
           domain[3]["name"], opt_iris.x_opt[3]))


Optimized Parameters:
	validation_split:	0.1
	batch_size:	10.0
	epochs:	20.0
	l1:	1024.0



In [0]:
print("optimized loss: {0}".format(opt_iris.fx_opt))

optimized loss: 0.10295619815587997


In [0]:
from sklearn import datasets
iris = datasets.load_iris()

In [0]:
from sklearn import tree        #importing subpackage from sklearn library
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.model_selection import GridSearchCV


In [0]:
criteria = ["gini", "entropy"]      #criteria to be tested
min_sample_split_range = [2,10, 20] #min sample split to be tested
max_depth_range = [None, 2, 5, 10]  #max depth to be tested
min_samples_leaf_range = [1, 5, 10] #min samples in the leaf to be tested
min_leaf_nodes_range = [None, 5, 10, 20]    #min leaf nodes to be tested

param_grid = {"criterion": criteria,
              "min_samples_split": min_sample_split_range,
              "max_depth": max_depth_range,
              "min_samples_leaf": min_samples_leaf_range,
              "max_leaf_nodes": min_leaf_nodes_range
                }

grid = GridSearchCV(estimator=tree.DecisionTreeClassifier(), 
                    param_grid=param_grid, 
                    cv = 5, 
                    scoring='accuracy', 
                    refit=True)     #setting grid with estimator

tree_model = make_pipeline(preprocessing.StandardScaler(), grid)    #creating preprocessing
tree_model.fit(iris.data, iris.target)      #fitting data

print("Accuracy of the tuned model: %.4f" %grid.best_score_)
print(grid.best_params_)

Accuracy of the tuned model: 0.9733
{'criterion': 'gini', 'max_depth': None, 'max_leaf_nodes': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}


In [0]:
from sklearn.neural_network import MLPClassifier

parameter_space = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (512,), (1024,)],
    'batch_size':[10, 50],
    'max_iter': [5, 10, 20]
}

In [0]:
from sklearn.model_selection import GridSearchCV
x = iris.data
y_ = iris.target.reshape(-1, 1) # Convert data to a single column
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)

In [0]:
mlp = MLPClassifier(solver = 'adam', validation_fraction = 0.1)
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'

In [0]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


Best parameters found:
 {'batch_size': 10, 'hidden_layer_sizes': (1024,), 'max_iter': 20}
0.308 (+/-0.201) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 5}
0.483 (+/-0.366) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (64,), 'max_iter': 20}
0.417 (+/-0.232) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 5}
0.658 (+/-0.062) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (128,), 'max_iter': 20}
0.433 (+/-0.309) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 5}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 10}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_sizes': (256,), 'max_iter': 20}
0.658 (+/-0.062) for {'batch_size': 10, 'hidden_layer_sizes': (512,), 'max_iter': 5}
0.683 (+/-0.047) for {'batch_size': 10, 'hidden_layer_siz

In [0]:
y_true, y_pred = Y_test , clf.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))


Results on the test set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.92      0.96        12
           2       1.00      0.75      0.86         8

   micro avg       1.00      0.90      0.95        30
   macro avg       1.00      0.89      0.94        30
weighted avg       1.00      0.90      0.94        30
 samples avg       0.90      0.90      0.90        30



In [0]:
print("Accuracy of the tuned model: %.4f" %clf.best_score_)
print(clf.best_params_)

Accuracy of the tuned model: 0.7500
{'batch_size': 10, 'hidden_layer_sizes': (1024,), 'max_iter': 20}


In [0]:
clf.score(X_train, Y_train)

0.9333333333333333

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
random = RandomizedSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
random.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...fle=True,
                                           solver='adam', tol=0.0001,
                                           validation_fraction=0.1,
                                

In [0]:
# Best paramete set
print('Best parameters found:\n', random.best_params_)

# All results
means = random.cv_results_['mean_test_score']
stds = random.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'max_iter': 20, 'hidden_layer_sizes': (1024,), 'batch_size': 10}
0.108 (+/-0.306) for {'max_iter': 5, 'hidden_layer_sizes': (1024,), 'batch_size': 50}
0.508 (+/-0.295) for {'max_iter': 5, 'hidden_layer_sizes': (256,), 'batch_size': 10}
0.108 (+/-0.306) for {'max_iter': 20, 'hidden_layer_sizes': (64,), 'batch_size': 50}
0.667 (+/-0.047) for {'max_iter': 20, 'hidden_layer_sizes': (256,), 'batch_size': 50}
0.667 (+/-0.047) for {'max_iter': 10, 'hidden_layer_sizes': (128,), 'batch_size': 10}
0.708 (+/-0.085) for {'max_iter': 20, 'hidden_layer_sizes': (1024,), 'batch_size': 10}
0.117 (+/-0.262) for {'max_iter': 20, 'hidden_layer_sizes': (128,), 'batch_size': 50}
0.458 (+/-0.366) for {'max_iter': 10, 'hidden_layer_sizes': (512,), 'batch_size': 50}
0.108 (+/-0.306) for {'max_iter': 5, 'hidden_layer_sizes': (256,), 'batch_size': 50}
0.125 (+/-0.147) for {'max_iter': 5, 'hidden_layer_sizes': (64,), 'batch_size': 10}


In [0]:
y_true, y_pred = Y_test , random.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.83      0.91        12
           2       1.00      1.00      1.00         8

   micro avg       1.00      0.93      0.97        30
   macro avg       1.00      0.94      0.97        30
weighted avg       1.00      0.93      0.96        30
 samples avg       0.93      0.93      0.93        30



In [0]:
random.score(X_train, Y_train)

0.9166666666666666